In [1]:
%run 'tree2seq_dataloader.ipynb'

  0%|          | 0/1000 [00:00<?, ?it/s]

71


Columns 0 to 12 
    0     1     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 26 to 38 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 39 to 51 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 52 to 64 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 65 to 70 
    0     0     0     0     0     0
[torch.FloatTensor of size 1x71]

(* (* (+ 55 (+ 56 53)) 31) (- 5 (* 54 9)))
( * ( * ( + 55 ( + 56 53 ) ) 31 ) ( - 5 ( * 54 9 ) ) )

(* (* (+ 55 (+ 56 53)) 31) (- 5 (* 54 9)))
['55', '56', '53', '31', '5', '54', '9']


100%|██████████| 1000/1000 [00:00<00:00, 3185.33it/s]

(12 (6 (4 0 (3 1 2)) 5) (11 7 (10 8 9)))
(* (* (+ 55 (+ 56 53)) 31) (- 5 (* 54 9)))


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable as Var

# module for childsumtreelstm
class ChildSumTreeLSTM(nn.Module):
    def __init__(self, in_dim, mem_dim):
        super(ChildSumTreeLSTM, self).__init__()
        self.in_dim = in_dim
        self.mem_dim = mem_dim
        self.ioux = nn.Linear(self.in_dim, 3 * self.mem_dim)
        self.iouh = nn.Linear(self.mem_dim, 3 * self.mem_dim)
        self.fx = nn.Linear(self.in_dim, self.mem_dim)
        self.fh = nn.Linear(self.mem_dim, self.mem_dim)

    def node_forward(self, inputs, child_c, child_h):
        child_h_sum = torch.sum(child_h, dim=0, keepdim=True)

        iou = self.ioux(inputs) + self.iouh(child_h_sum)
        i, o, u = torch.split(iou, iou.size(1) // 3, dim=1)
        i, o, u = F.sigmoid(i), F.sigmoid(o), F.tanh(u)

        f = F.sigmoid(
            self.fh(child_h) +
            self.fx(inputs).repeat(len(child_h), 1)
        )
        fc = torch.mul(f, child_c)

        c = torch.mul(i, u) + torch.sum(fc, dim=0, keepdim=True)
        h = torch.mul(o, F.tanh(c))
        return c, h

    def forward(self, tree, inputs):
        if isinstance(tree, Tree):
            child_states = []
            for child in tree:
                child_states.append(self.forward(child, inputs))
                
            child_c, child_h = zip(* map(lambda x: x, child_states))
            child_c, child_h = torch.cat(child_c, dim=0), torch.cat(child_h, dim=0)
            state = self.node_forward(inputs[tree.label()], child_c, child_h)
        else:
            child_c = Var(inputs[0].data.new(1, self.mem_dim).fill_(0.))
            child_h = Var(inputs[0].data.new(1, self.mem_dim).fill_(0.))
            state = self.node_forward(inputs[tree], child_c, child_h)
        return state


In [4]:
if __name__ == '__main__':
    print(dir())
    dataset = Dataset()
    dataset.create_vocab('data/train.orig')
    one_hot_dict = dataset.create_one_hot()

    trees = dataset.read_trees('data/train.orig')
    seqs = dataset.read_seqs('data/train.orig')

100%|██████████| 1000/1000 [00:00<00:00, 435138.91it/s]

['ChildSumTreeLSTM', 'Dataset', 'F', 'In', 'Out', 'Tree', 'Var', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__name__', '__package__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_ih', '_ii', '_iii', '_oh', '_sh', 'copy', 'data', 'dataset', 'exit', 'get_ipython', 'nn', 'np', 'one_hot_dict', 'operator', 'ptr_trees', 'quit', 'seqs', 'tmp', 'torch', 'tqdm', 'tree', 'trees']
71


In [5]:
cst = ChildSumTreeLSTM(dataset.vector_dim, 64)

In [6]:
cst.forward(ptr_trees[1][0], Var(ptr_trees[1][1]))

(Variable containing:
 
 Columns 0 to 9 
  0.0732  0.0741 -0.2140  0.1343 -0.4944  0.2524 -0.3643  0.0967  0.1445 -0.1956
 
 Columns 10 to 19 
  0.1266  0.0453  0.0874 -0.1592 -0.2266 -0.1153 -0.0979  0.3408  0.1792  0.1575
 
 Columns 20 to 29 
 -0.1935 -0.0367  0.0714 -0.1241 -0.0218 -0.2122  0.2396  0.1159  0.0614 -0.0815
 
 Columns 30 to 39 
  0.2067 -0.1138 -0.0770  0.5037  0.2422  0.2934  0.0545  0.0192 -0.1438 -0.2762
 
 Columns 40 to 49 
  0.1492 -0.0113 -0.2250 -0.2761  0.0296 -0.0191  0.1368 -0.1910  0.0893 -0.0686
 
 Columns 50 to 59 
 -0.1498 -0.0770  0.1058 -0.0612 -0.2940 -0.0334  0.1732  0.1049 -0.3903  0.3993
 
 Columns 60 to 63 
 -0.0513 -0.2406 -0.1352  0.2209
 [torch.FloatTensor of size 1x64], Variable containing:
 
 Columns 0 to 9 
  0.0420  0.0401 -0.1166  0.0597 -0.2233  0.1380 -0.1563  0.0446  0.0709 -0.1104
 
 Columns 10 to 19 
  0.0573  0.0203  0.0455 -0.0788 -0.1053 -0.0471 -0.0490  0.1862  0.0948  0.0828
 
 Columns 20 to 29 
 -0.1006 -0.0169  0.0319 -0.0596 -0